In [1]:
nope = 128
rope = 64

In [2]:
import torch
queries = torch.randn(1, 1, 8, 192)

In [ ]:
import torch
queries = torch.randn(1, 1, 8, 192)
q_nope, q_rope = torch.split(queries, [nope, rope], dim=-1)
q_nope.shape, q_rope.shape

(torch.Size([1, 1, 8, 128]), torch.Size([1, 1, 8, 64]))

In [ ]:
queries = torch.randn(1, 1, 8, 192)
keys = torch.randn(1, 1, 8, 192)
values = torch.randn(1, 1, 8, 128)